# Statistical Analysis STAR-RIS Assisted CoMP-NOMA

In [ ]:
import numpy as np
import scipy as sp
import mpmath as mpm
from scipy.special import gamma
import sympy as sym
from sympy import meijerg
from sympy import N as NEval
import matplotlib.pyplot as plt
import simcomm.core.propagation as prop
from simcomm.core import Nakagami, get_rvs
from simcomm.utils import dbm2pow, db2pow, pow2db, pow2dbm, get_distance, qfunc
from simcomm.core.propagation import get_noise_power, get_pathloss

In [ ]:
environment = {
    "positions": {
        "BS1": [-50, 0, 25],  # BS1 position
        "BS2": [50, 0, 25],  # BS2 position
        "RIS": [0, 25, 5],  # RIS position
        "Uf": [0, 35, 1],  # Uf position
        "U1c": [-40, 18, 1],  # U1c position
        "U2c": [30, 22, 1],  # U2c position
    },
    "fading": {
        "rayleigh": {"type": "rayleigh", "sigma": 1},  # Rayleigh fading
        "ricianE": {
            "type": "rician",
            "K": 4,
            "sigma": 1,
        },  # Rician fading for edge users
        "ricianC": {
            "type": "rician",
            "K": 3,
            "sigma": 1,
        },  # Rician fading for center users
        "nakagami-id": {
            "type": "nakagami",
            "m": 1,
            "omega": 1,
        },  # Nakagami-m fading with m = 1
        "nakagami-d": {
            "type": "nakagami",
            "m": 3,
            "omega": 3**2,
        },
    },  # Nakagami-m fading with m = 2
    "pathloss": {
        "center": {"type": "free-space", "alpha": 3, "p0": 30},  # Center users
        "ris": {"type": "free-space", "alpha": 3, "p0": 30},  # BS to RIS
        "risC": {"type": "free-space", "alpha": 2.7, "p0": 30},  # RIS to center users
        "risE": {"type": "free-space", "alpha": 2.3, "p0": 30},  # RIS to edge user
        "edge": {"type": "free-space", "alpha": 3.5, "p0": 30},  # Edge users
        "inter": {"type": "free-space", "alpha": 3.5, "p0": 30},  # Interference links
    },
}

setting = {
    "ris32": {
        "ris_enhanced": True,  # RIS with 32 elements
        "ris_elements": 32,
        "comp_enabled": True,  # CoMP enabled
    },
    "no_ris": {
        "ris_enhanced": False,  # No RIS
        "comp_enabled": True,  # CoMP enabled"
    },
    "ris70": {
        "ris_enhanced": True,  # RIS with 70 elements
        "ris_elements": 70,
        "comp_enabled": True,  # CoMP enabled
    },
    "no_ris_non_comp": {
        "ris_enhanced": False,  # No RIS
        "comp_enabled": False,  # CoMP disabled
    },
}

constants = {
    "BANDWIDTH": 1e6,  # Bandwidth in Hz
    "TEMP": 300,  # Temperature in Kelvin
    "FREQ": 2.4e9,  # Frequency of carrier signal in Hz
    "SIGMA": 6.32,  # Shadowing standard deviation in dB
}

# Load the environment
pathloss_cfg = environment["pathloss"]
fading_cfg = environment["fading"]
positions = environment["positions"]

# Additional parameters
BANDWIDTH = constants["BANDWIDTH"]  # Bandwidth in Hz
TEMP = constants["TEMP"]  # Temperature in Kelvin
FREQ = constants["FREQ"]  # Frequency of carrier signal in Hz
SIGMA = constants["SIGMA"]  # Shadowing standard deviation in dB

In [ ]:
# Positions
pos_BS1 = positions["BS1"]
pos_BS2 = positions["BS2"]
pos_RIS = positions["RIS"]
pos_U1c = positions["U1c"]
pos_U2c = positions["U2c"]
pos_Uf = positions["Uf"]

# Distance parameters
## BS1 specific distances
distance_iR = get_distance(pos_BS1, pos_RIS, 3)
distance_Rc = get_distance(pos_RIS, pos_U1c, 3)
distance_ic = get_distance(pos_BS1, pos_U1c, 3)
distance_if = get_distance(pos_BS1, pos_Uf, 3)

## BS2 specific distances
distance_iR_hat = get_distance(pos_BS2, pos_RIS, 3)
distance_if_hat = get_distance(pos_BS2, pos_Uf, 3)

## Common distances
distance_Rf = get_distance(pos_RIS, pos_Uf, 3)
distance_ic_prime = get_distance(pos_BS2, pos_U1c, 3)

## BS specific distances for U2
distance_ic_u2 = get_distance(pos_BS2, pos_U2c, 3)
distance_Rc_u2 = get_distance(pos_RIS, pos_U2c, 3)
distance_ic_prime_u2 = get_distance(pos_BS1, pos_U2c, 3)


# #################### CUSTOM PARAMETERS ####################
# distance_Rf = 3
# distance_ic_prime = 50
# distance_ic_u2 = 25
# distance_Rc_u2 = 10
# distance_ic_prime_u2 = 50
# distance_iR = 25
# distance_Rc = 10
# distance_ic = 25
# distance_if = 40
# distance_iR_hat = 25
# distance_if_hat = 40

# Simulation parameters
K = 70
N = 10000
beta_r = 0.5
beta_t = 0.5
shape = (N, 1)
shape_ris = (K // 2, N, 1)

# Additional parameters
BANDWIDTH = constants["BANDWIDTH"]  # Bandwidth in Hz
TEMP = constants["TEMP"]  # Temperature in Kelvin
FREQ = constants["FREQ"]  # Frequency of carrier signal in Hz
SIGMA = constants["SIGMA"]  # Shadowing standard deviation in dB

Pt = np.linspace(-40, 20, 121)  # Transmit power in dBm
Pt_lin = dbm2pow(Pt)  # Transmit power in linear scale
N0 = get_noise_power(BANDWIDTH, TEMP, 12)  # Noise power in dBm
N0_lin = dbm2pow(N0)  # Noise power in linear scale

# Power allocation
zeta_ic = 0.3
zeta_if = 0.7
zeta_ic_hat = 0.3
zeta_if_hat = 0.7

rho = Pt_lin / N0_lin  # SNR
scale_ic = np.zeros_like(rho)
scale_ic_prime = np.linspace(0.001, 1.4, len(rho) - 20)
scale_ic[20:] = scale_ic_prime
scale_f = np.linspace(0.01, 0.15, len(rho))

## Simulation

In [ ]:
pathloss_iR = get_pathloss(**pathloss_cfg["ris"], distance=distance_iR, frequency=FREQ)
pathloss_Rc = get_pathloss(**pathloss_cfg["risC"], distance=distance_Rc, frequency=FREQ)
pathloss_ic = get_pathloss(
    **pathloss_cfg["center"], distance=distance_ic, frequency=FREQ
)
pathloss_ic_prime = get_pathloss(
    **pathloss_cfg["inter"], distance=distance_ic_prime, frequency=FREQ
)
pathloss_ic_u2 = get_pathloss(
    **pathloss_cfg["center"], distance=distance_ic_u2, frequency=FREQ
)
pathloss_ic_prime_u2 = get_pathloss(
    **pathloss_cfg["inter"], distance=distance_ic_prime_u2, frequency=FREQ
)
pathloss_Rc_u2 = get_pathloss(
    **pathloss_cfg["risC"], distance=distance_Rc_u2, frequency=FREQ
)
pathloss_if = get_pathloss(**pathloss_cfg["edge"], distance=distance_if, frequency=FREQ)
pathloss_iR_hat = get_pathloss(
    **pathloss_cfg["ris"], distance=distance_iR_hat, frequency=FREQ
)
pathloss_if_hat = get_pathloss(
    **pathloss_cfg["edge"], distance=distance_if_hat, frequency=FREQ
)
pathloss_Rf = get_pathloss(**pathloss_cfg["risE"], distance=distance_Rf, frequency=FREQ)

samples_iR = get_rvs(**fading_cfg["nakagami-d"], shape=shape_ris)
samples_Rc = get_rvs(**fading_cfg["nakagami-d"], shape=shape_ris)
samples_ic = get_rvs(**fading_cfg["nakagami-id"], shape=shape)
samples_ic_prime = get_rvs(**fading_cfg["nakagami-id"], shape=shape)

samples_if = get_rvs(**fading_cfg["nakagami-id"], shape=shape)
samples_iR_hat = get_rvs(**fading_cfg["nakagami-d"], shape=shape_ris)
samples_if_hat = get_rvs(**fading_cfg["nakagami-id"], shape=shape)
samples_Rf = get_rvs(**fading_cfg["nakagami-d"], shape=shape_ris)

In [ ]:
h_iR = np.abs(np.sqrt(db2pow(-1 * pathloss_iR)) * samples_iR)
h_Rc = np.abs(np.sqrt(db2pow(-1 * pathloss_Rc)) * samples_Rc)
h_ic = np.abs(np.sqrt(db2pow(-1 * pathloss_ic)) * samples_ic)
h_ic_prime = np.abs(np.sqrt(db2pow(-1 * pathloss_ic_prime)) * samples_ic_prime)
h_if = np.abs(np.sqrt(db2pow(-1 * pathloss_if)) * samples_if)
h_iR_hat = np.abs(np.sqrt(db2pow(-1 * pathloss_iR_hat)) * samples_iR_hat)
h_if_hat = np.abs(np.sqrt(db2pow(-1 * pathloss_if_hat)) * samples_if_hat)
h_Rf = np.abs(np.sqrt(db2pow(-1 * pathloss_Rf)) * samples_Rf)
h_ic_u2 = np.abs(np.sqrt(db2pow(-1 * pathloss_ic_u2)) * samples_ic)
h_ic_prime_u2 = np.abs(np.sqrt(db2pow(-1 * pathloss_ic_prime_u2)) * samples_ic_prime)
h_Rc_u2 = np.abs(np.sqrt(db2pow(-1 * pathloss_Rc_u2)) * samples_Rc)

cascaded_ic = np.sum(h_iR * h_Rc, axis=0) * np.sqrt(beta_r)
H_ic = cascaded_ic + h_ic

cascaded_if = np.sum(h_iR * h_Rf, axis=0) * np.sqrt(beta_t)
cascaded_if_hat = np.sum(h_iR_hat * h_Rf, axis=0) * np.sqrt(beta_t)
H_if = cascaded_if + h_if
H_if_hat = cascaded_if_hat + h_if_hat

cascaded_ic_u2 = np.sum(h_iR_hat * h_Rc_u2, axis=0) * np.sqrt(beta_r)
H_ic_u2 = cascaded_ic_u2 + h_ic_u2

In [ ]:
print(np.mean(cascaded_if))
print(np.mean(h_if))
print(np.mean(H_if))

## SINR PDFs

### PDF of SINR $\gamma_{i,c\rightarrow f}$

Effective Channel $\textbf{H}_{i, c}=\textbf{h}_{i, c}+\textbf{h}_{R, c}^H \mathbf{\Theta_r}\textbf{h}_{i, R}$

Channel Gain $Z_{i,c} = |H_{i,c}|^2 = (h_{i,c} + G_{i,R,c})^2$

Direct Channel $h_{i,c}$

In [ ]:
m_ic = 1
omega_ic = 2 * db2pow(-1 * pathloss_ic)
omega_ic_prime = 2 * db2pow(-1 * pathloss_ic_prime)

omega_ic_u2 = 2 * db2pow(-1 * pathloss_ic_u2)
omega_ic_prime_u2 = 2 * db2pow(-1 * pathloss_ic_prime_u2)

In [ ]:
def fun_mu_h_ic(p, m_ic, omega_ic):
    return (gamma(m_ic + p / 2) / gamma(m_ic)) * (m_ic / omega_ic) ** (-p / 2)

In [ ]:
mu_h_ic = fun_mu_h_ic(1, m_ic, omega_ic)  # 1st moment of h_ic
mu_h_ic_2 = fun_mu_h_ic(2, m_ic, omega_ic)  # 2nd moment of h_ic

mu_h_ic_u2 = fun_mu_h_ic(1, m_ic, omega_ic_u2)  # 1st moment of h_ic
mu_h_ic_2_u2 = fun_mu_h_ic(2, m_ic, omega_ic_u2)  # 2nd moment of h_ic

k_h_ic = (mu_h_ic**2) / (mu_h_ic_2 - mu_h_ic**2)  # Shape parameter of h_ic
theta_h_ic = (mu_h_ic_2 - mu_h_ic**2) / mu_h_ic  # Scale parameter of h_ic

k_h_ic_u2 = (mu_h_ic_u2**2) / (mu_h_ic_2_u2 - mu_h_ic_u2**2)  # Shape parameter of h_ic
theta_h_ic_u2 = (mu_h_ic_2_u2 - mu_h_ic_u2**2) / mu_h_ic_u2  # Scale parameter of h_ic

Cascaded Channel $G_{i,R,c} = \sqrt{\beta_r} \sum_{k=1}^{K} |{h_{R,c}}||{h_{i,R}}|$

In [ ]:
m_iR = 2
m_Rc = 2
omega_iR = 2 * db2pow(-1 * pathloss_iR)
omega_Rc = 2 * db2pow(-1 * pathloss_Rc)
omega_Rc_u2 = 2 * db2pow(-1 * pathloss_Rc_u2)

In [ ]:
def fun_mu_h_iR(p, m_iR, omega_iR):
    return gamma(m_iR + p / 2) / gamma(m_iR) * (m_iR / omega_iR) ** (-p / 2)


def fun_mu_h_Rc(p, m_Rc, omega_Rc):
    return gamma(m_Rc + p / 2) / gamma(m_Rc) * (m_Rc / omega_Rc) ** (-p / 2)

In [ ]:
# def fun_mu_G_iRc(p, m_iR, omega_iR, m_Rc, omega_Rc, K, beta_r):
#     mu = (gamma(m_iR + (1 / 2)) * gamma(m_Rc + (1 / 2))) / (
#         gamma(m_iR) * gamma(m_Rc) * (m_iR * m_Rc) ** (1 / 2)
#     )
#     k = mu**2 / (1 - mu**2)
#     theta = (1 - mu**2) / mu

#     return (
#         gamma((K * k) + p) * (np.sqrt(omega_iR * omega_Rc * beta_r) * theta) ** p
#     ) / gamma(K * k)


def fun_mu_G_iRc(p, m_iR, omega_iR, m_Rc, omega_Rc, K, beta_r):
    return (
        gamma(m_Rc + (p / 2))
        * (np.sqrt(beta_r) * K) ** p
        * gamma(m_iR + (p / 2))
        * ((m_iR * m_Rc) / (omega_iR * omega_Rc)) ** (-p / 2)
    ) / (gamma(m_iR) * gamma(m_Rc))


mu_G_iRc = fun_mu_G_iRc(
    1, m_iR, omega_iR, m_Rc, omega_Rc, K // 2, beta_r
)  # 1st moment of G_iRc
mu_G_iRc_2 = fun_mu_G_iRc(
    2, m_iR, omega_iR, m_Rc, omega_Rc, K // 2, beta_r
)  # 2nd moment of G_iRc

mu_G_iRc_u2 = fun_mu_G_iRc(
    1, m_iR, omega_iR, m_Rc, omega_Rc_u2, K // 2, beta_r
)  # 1st moment of G_iRc
mu_G_iRc_2_u2 = fun_mu_G_iRc(
    2, m_iR, omega_iR, m_Rc, omega_Rc_u2, K // 2, beta_r
)  # 2nd moment of G_iRc


k_G_iRc = mu_G_iRc**2 / (mu_G_iRc_2 - mu_G_iRc**2)  # Shape parameter of G_iRc
theta_G_iRc = (mu_G_iRc_2 - mu_G_iRc**2) / mu_G_iRc  # Scale parameter of G_iRc

k_G_iRc_u2 = mu_G_iRc_u2**2 / (
    mu_G_iRc_2_u2 - mu_G_iRc_u2**2
)  # Shape parameter of G_iRc
theta_G_iRc_u2 = (
    mu_G_iRc_2_u2 - mu_G_iRc_u2**2
) / mu_G_iRc_u2  # Scale parameter of G_iRc

Effective Channel Gain $Z_{i,c} = |H_{i,c}|^2 = (h_{i,c} + G_{i,R,c})^2$

In [ ]:
mu_Z_ic = (
    fun_mu_G_iRc(2, m_iR, omega_iR, m_Rc, omega_Rc, K // 2, beta_r)
    + fun_mu_h_ic(2, m_ic, omega_ic)
    + (
        2
        * fun_mu_G_iRc(1, m_iR, omega_iR, m_Rc, omega_Rc, K // 2, beta_r)
        * fun_mu_h_ic(1, m_ic, omega_ic)
    )
)

mu_Z_ic_2 = (
    fun_mu_G_iRc(4, m_iR, omega_iR, m_Rc, omega_Rc, K // 2, beta_r)
    + fun_mu_h_ic(4, m_ic, omega_ic)
    + (
        6
        * fun_mu_G_iRc(2, m_iR, omega_iR, m_Rc, omega_Rc, K // 2, beta_r)
        * fun_mu_h_ic(2, m_ic, omega_ic)
    )
    + (
        4
        * fun_mu_h_ic(3, m_ic, omega_ic)
        * fun_mu_G_iRc(1, m_iR, omega_iR, m_Rc, omega_Rc, K // 2, beta_r)
    )
    + (
        4
        * fun_mu_h_ic(1, m_ic, omega_ic)
        * fun_mu_G_iRc(3, m_iR, omega_iR, m_Rc, omega_Rc, K // 2, beta_r)
    )
)

mu_Z_ic_u2 = (
    fun_mu_G_iRc(2, m_iR, omega_iR, m_Rc, omega_Rc_u2, K // 2, beta_r)
    + fun_mu_h_ic(2, m_ic, omega_ic_u2)
    + (
        2
        * fun_mu_G_iRc(1, m_iR, omega_iR, m_Rc, omega_Rc_u2, K // 2, beta_r)
        * fun_mu_h_ic(1, m_ic, omega_ic_u2)
    )
)

mu_Z_ic_2_u2 = (
    fun_mu_G_iRc(4, m_iR, omega_iR, m_Rc, omega_Rc_u2, K // 2, beta_r)
    + fun_mu_h_ic(4, m_ic, omega_ic_u2)
    + (
        6
        * fun_mu_G_iRc(2, m_iR, omega_iR, m_Rc, omega_Rc_u2, K // 2, beta_r)
        * fun_mu_h_ic(2, m_ic, omega_ic_u2)
    )
    + (
        4
        * fun_mu_h_ic(3, m_ic, omega_ic_u2)
        * fun_mu_G_iRc(1, m_iR, omega_iR, m_Rc, omega_Rc_u2, K // 2, beta_r)
    )
    + (
        4
        * fun_mu_h_ic(1, m_ic, omega_ic_u2)
        * fun_mu_G_iRc(3, m_iR, omega_iR, m_Rc, omega_Rc_u2, K // 2, beta_r)
    )
)

k_Z_ic = mu_Z_ic**2 / (mu_Z_ic_2 - mu_Z_ic**2)  # Shape parameter of Z_ic
theta_Z_ic = (mu_Z_ic_2 - mu_Z_ic**2) / mu_Z_ic  # Scale parameter of Z_ic

k_Z_ic_u2 = mu_Z_ic_u2**2 / (
    mu_Z_ic_2_u2 - mu_Z_ic_u2**2
)  # Shape parameter of Z_ic
theta_Z_ic_u2 = (mu_Z_ic_2_u2 - mu_Z_ic_u2**2) / mu_Z_ic_u2  # Scale parameter of Z_ic

$\mathcal{V_{i,c,f}} = {\zeta_{i,f}\rho|\textbf{H}_{i,c}|^2},$

where $\rho = \frac{P}{\sigma^2}$

In [ ]:
simulation_V_icf = zeta_if * rho * H_ic**2
simulation_V_icf_u2 = zeta_if * rho * H_ic_u2**2

In [ ]:
k_V_icf = k_Z_ic * np.ones_like(rho)
theta_V_icf = theta_Z_ic * zeta_if * rho

k_V_icf_u2 = k_Z_ic_u2 * np.ones_like(rho)
theta_V_icf_u2 = theta_Z_ic_u2 * zeta_if * rho

$\mathcal{B_{i,c,f}} = {\zeta_{i,c}\rho|\textbf{H}_{i,c}|^2 + \rho|\textbf{h}_{i,c^\prime}|^2 +  1}$

$\mathcal{B_{i,c,f}} = {\mathcal{W_{i,c}} +  1}$,

where $\mathcal{W_{i,c,f}} = \zeta_{i,c}\rho|\textbf{H}_{i,c}|^2 + \rho|\textbf{h}_{i,c^\prime}|^2$

In [ ]:
simulation_W_icf = (zeta_ic * rho * H_ic**2) + (rho * h_ic_prime**2)
simulation_B_icf = simulation_W_icf + 1

simulation_W_icf_u2 = (zeta_ic * rho * H_ic_u2**2) + (rho * h_ic_prime_u2**2)
simulation_B_icf_u2 = simulation_W_icf_u2 + 1

$\mu _{\mathcal{W}_{ic}} = \rho  \mu _{h_{ic}}+\zeta  \rho  \mu _{Z_{ic}}$

$\mu _{\mathcal{W}_{ic}}^{(2)} = \rho ^2 \left(\zeta  \left(2 \mu _{h_{ic}} \mu
   _{Z_{ic}}+\zeta  \mu _{Z_{\text{ic2}}}\right)+\mu
   _{h_{\text{ic2}}}\right)$

In [ ]:
def fun_mu_h_ic_sqr_interf(p, m_ic, omega_ic):
    return (gamma(m_ic + p) / gamma(m_ic)) * (m_ic / omega_ic) ** (-p)

In [ ]:
mu_W_icf = (
    rho * fun_mu_h_ic_sqr_interf(1, m_ic, omega_ic_prime) + zeta_ic * rho * mu_Z_ic
)  # 1st moment of W_ic

mu_W_icf_2 = rho**2 * (
    zeta_ic
    * (
        (2 * fun_mu_h_ic_sqr_interf(1, m_ic, omega_ic_prime) * mu_Z_ic)
        + zeta_ic * mu_Z_ic_2
    )
    + fun_mu_h_ic_sqr_interf(2, m_ic, omega_ic_prime)
)  # 2nd moment of W_ic

mu_W_icf_u2 = (
    rho * fun_mu_h_ic_sqr_interf(1, m_ic, omega_ic_prime_u2)
    + zeta_ic * rho * mu_Z_ic_u2
)  # 1st moment of W_ic

mu_W_icf_2_u2 = rho**2 * (
    zeta_ic
    * (
        (2 * fun_mu_h_ic_sqr_interf(1, m_ic, omega_ic_prime_u2) * mu_Z_ic_u2)
        + zeta_ic * mu_Z_ic_2_u2
    )
    + fun_mu_h_ic_sqr_interf(2, m_ic, omega_ic_prime_u2)
)  # 2nd moment of W_ic


k_W_icf = mu_W_icf**2 / (mu_W_icf_2 - mu_W_icf**2)  # Shape parameter of W_ic
theta_W_icf = (mu_W_icf_2 - mu_W_icf**2) / mu_W_icf  # Scale parameter of W_ic

k_W_icf_u2 = mu_W_icf_u2**2 / (
    mu_W_icf_2_u2 - mu_W_icf_u2**2
)  # Shape parameter of W_ic
theta_W_icf_u2 = (
    mu_W_icf_2_u2 - mu_W_icf_u2**2
) / mu_W_icf_u2  # Scale parameter of W_ic

In [ ]:
mu_B_icf = mu_W_icf + 1  # 1st moment of B_icf
mu_B_icf_2 = mu_W_icf_2 + 2 * mu_W_icf + 1  # 2nd moment of B_icf

mu_B_icf_u2 = mu_W_icf_u2 + 1  # 1st moment of B_icf
mu_B_icf_2_u2 = mu_W_icf_2_u2 + 2 * mu_W_icf_u2 + 1  # 2nd moment of B_icf

k_B_icf = mu_B_icf**2 / (mu_B_icf_2 - mu_B_icf**2)  # Shape parameter of B_icf
theta_B_icf = (mu_B_icf_2 - mu_B_icf**2) / mu_B_icf  # Scale parameter of B_icf

k_B_icf_u2 = mu_B_icf_u2**2 / (
    mu_B_icf_2_u2 - mu_B_icf_u2**2
)  # Shape parameter of B_icf
theta_B_icf_u2 = (
    mu_B_icf_2_u2 - mu_B_icf_u2**2
) / mu_B_icf_u2  # Scale parameter of B_icf

In [ ]:
simulation_SINR_icf = simulation_V_icf / simulation_B_icf
simulation_SINR_icf_u2 = simulation_V_icf_u2 / simulation_B_icf_u2

In [ ]:
def generalized_beta_prime_pdf(x, a, b, p, q):
    return (p * ((x / q) ** (a * p - 1)) * (1 + (x / q) ** p) ** (-a - b)) / (
        q * sp.special.beta(a, b)
    )


def hypergeometric_f1_regularized(a, b, c):
    return sp.special.hyp1f1(a, b, c) / sp.special.gamma(b)

### PDF of SINR $\gamma_{i,c}$

$\mathcal{V_{i,c}} = {\zeta_{i,c}\rho|\textbf{H}_{i,c}|^2},$

where $\rho = \frac{P}{\sigma^2}$

In [ ]:
simulation_V_ic = zeta_ic * rho * H_ic**2
simulation_V_ic_u2 = zeta_ic * rho * H_ic_u2**2

In [ ]:
k_V_ic = k_Z_ic * np.ones_like(rho)
theta_V_ic = theta_Z_ic * zeta_ic * rho

k_V_ic_u2 = k_Z_ic_u2 * np.ones_like(rho)
theta_V_ic_u2 = theta_Z_ic_u2 * zeta_ic * rho

$\mathcal{B_{i,c}} = {\rho|\textbf{h}_{i,c^\prime}|^2 +  1}$

$\mathcal{B_{i,c}} = {\mathcal{W_{i,c}} +  1}$,

where $\mathcal{W_{i,c}} = \rho|\textbf{h}_{i,c^\prime}|^2$

In [ ]:
simulation_B_ic = (rho * h_ic_prime**2) + 1
simulation_B_ic_u2 = (rho * h_ic_prime_u2**2) + 1

In [ ]:
def fun_mu_h_ic_sqr_interf(p, m_ic, omega_ic):
    return (gamma(m_ic + p) / gamma(m_ic)) * (m_ic / omega_ic) ** (-p)

In [ ]:
mu_W_ic = rho * fun_mu_h_ic_sqr_interf(1, m_ic, omega_ic_prime)  # 1st moment of W_ic
mu_W_ic_2 = rho**2 * fun_mu_h_ic_sqr_interf(
    2, m_ic, omega_ic_prime
)  # 2nd moment of W_ic

mu_W_ic_u2 = rho * fun_mu_h_ic_sqr_interf(
    1, m_ic, omega_ic_prime_u2
)  # 1st moment of W_ic
mu_W_ic_2_u2 = rho**2 * fun_mu_h_ic_sqr_interf(
    2, m_ic, omega_ic_prime_u2
)  # 2nd moment of W_ic

k_W_ic = mu_W_ic**2 / (mu_W_ic_2 - mu_W_ic**2)  # Shape parameter of W_ic
theta_W_ic = (mu_W_ic_2 - mu_W_ic**2) / mu_W_ic  # Scale parameter of W_ic

k_W_ic_u2 = mu_W_ic_u2**2 / (
    mu_W_ic_2_u2 - mu_W_ic_u2**2
)  # Shape parameter of W_ic
theta_W_ic_u2 = (mu_W_ic_2_u2 - mu_W_ic_u2**2) / mu_W_ic_u2  # Scale parameter of W_ic

In [ ]:
mu_B_ic = mu_W_ic + 1  # 1st moment of B_ic
mu_B_ic_2 = mu_W_ic_2 + 2 * mu_W_ic + 1  # 2nd moment of B_ic

mu_B_ic_u2 = mu_W_ic_u2 + 1  # 1st moment of B_ic
mu_B_ic_2_u2 = mu_W_ic_2_u2 + 2 * mu_W_ic_u2 + 1  # 2nd moment of B_ic

k_B_ic = mu_B_ic**2 / (mu_B_ic_2 - mu_B_ic**2)  # Shape parameter of B_ic
theta_B_ic = (mu_B_ic_2 - mu_B_ic**2) / mu_B_ic  # Scale parameter of B_ic

k_B_ic_u2 = mu_B_ic_u2**2 / (
    mu_B_ic_2_u2 - mu_B_ic_u2**2
)  # Shape parameter of B_ic
theta_B_ic_u2 = (mu_B_ic_2_u2 - mu_B_ic_u2**2) / mu_B_ic_u2  # Scale parameter of B_ic

In [ ]:
simulation_SINR_ic = simulation_V_ic / simulation_B_ic
simulation_SINR_ic_u2 = simulation_V_ic_u2 / simulation_B_ic_u2

### PDF of SINR $\gamma_{f}$

Effective Channel $\textbf{H}_{i, f}=\textbf{h}_{i, f}+\textbf{h}_{R, f}^H \mathbf{\Theta_r}\textbf{h}_{i, R}$

Channel Gain $Z_{i,f} = |H_{i,f}|^2 = (h_{i,f} + G_{i,R,f})^2$

Direct Channel $h_{i,f}$

Additionally, $\hat{i} \in \mathcal{I}_c \backslash i$

In [ ]:
m_if = 1
m_if_hat = 1
omega_if = 2 * db2pow(-1 * pathloss_if)
omega_if_hat = 2 * db2pow(-1 * pathloss_if_hat)

In [ ]:
def fun_mu_h_if(p, m_if, omega_if):
    return (gamma(m_if + p / 2) / gamma(m_if)) * (m_if / omega_if) ** (-p / 2)

In [ ]:
mu_h_if = fun_mu_h_if(1, m_if, omega_if)  # 1st moment of h_if
mu_h_if_2 = fun_mu_h_if(2, m_if, omega_if)  # 2nd moment of h_if
mu_h_if_hat = fun_mu_h_if(1, m_if_hat, omega_if_hat)  # 1st moment of h_if_hat
mu_h_if_hat_2 = fun_mu_h_if(2, m_if_hat, omega_if_hat)  # 2nd moment of h_if_hat

k_h_if = (mu_h_if**2) / (mu_h_if_2 - mu_h_if**2)  # Shape parameter of h_if
theta_h_if = (mu_h_if_2 - mu_h_if**2) / mu_h_if  # Scale parameter of h_if

k_h_if_hat = (mu_h_if_hat**2) / (
    mu_h_if_hat_2 - mu_h_if_hat**2
)  # Shape parameter of h_if_hat
theta_h_if_hat = (
    mu_h_if_hat_2 - mu_h_if_hat**2
) / mu_h_if_hat  # Scale parameter of h_if_hat

Cascaded Channel $G_{i,R,f} = \sqrt{\beta_t} \sum_{k=1}^{K} |{h_{R,f}}||{h_{i,R}}|$

In [ ]:
m_iR = 2
m_iR_hat = 2
m_Rf = 2
omega_iR = 2 * db2pow(-1 * pathloss_iR)
omega_iR_hat = 2 * db2pow(-1 * pathloss_iR_hat)
omega_Rf = 2 * db2pow(-1 * pathloss_Rf)

In [ ]:
def fun_mu_h_iR(p, m_iR, omega_iR):
    return gamma(m_iR + p / 2) / gamma(m_iR) * (m_iR / omega_iR) ** (-p / 2)


def fun_mu_h_Rf(p, m_Rf, omega_Rf):
    return gamma(m_Rf + p / 2) / gamma(m_Rf) * (m_Rf / omega_Rf) ** (-p / 2)

In [ ]:
# def fun_mu_G_iRf(p, m_iR_hat, omega_iR_hat, m_Rf, omega_Rf, K, beta_t):
#     mu = (gamma(m_iR_hat + (1 / 2)) * gamma(m_Rf + (1 / 2))) / (
#         gamma(m_iR_hat) * gamma(m_Rf) * (m_iR_hat * m_Rf) ** (1 / 2)
#     )
#     k = mu**2 / (1 - mu**2)
#     theta = (1 - mu**2) / mu

#     return (
#         gamma((K * k) + p) * (np.sqrt(omega_iR_hat * omega_Rf * beta_t) * theta) ** p
#     ) / gamma(K * k)


def fun_mu_G_iRf(p, m_iR, omega_iR, m_Rf, omega_Rf, K, beta_t):
    return (
        gamma(m_Rf + (p / 2))
        * (np.sqrt(beta_t) * K) ** p
        * gamma(m_iR + (p / 2))
        * ((m_iR * m_Rf) / (omega_iR * omega_Rf)) ** (-p / 2)
    ) / (gamma(m_iR) * gamma(m_Rf))


mu_G_iRf = fun_mu_G_iRf(
    1, m_iR, omega_iR, m_Rf, omega_Rf, K // 2, beta_t
)  # 1st moment of G_iRc
mu_G_iRf_2 = fun_mu_G_iRf(
    2, m_iR, omega_iR, m_Rf, omega_Rf, K // 2, beta_t
)  # 2nd moment of G_iRc

mu_G_iRf_hat = fun_mu_G_iRf(
    1, m_iR_hat, omega_iR_hat, m_Rf, omega_Rf, K // 2, beta_t
)  # 1st moment of G_iRc
mu_G_iRf_hat_2 = fun_mu_G_iRf(
    2, m_iR_hat, omega_iR_hat, m_Rf, omega_Rf, K // 2, beta_t
)  # 2nd moment of G_iRcarameter of G_iRc

k_G_iRf = mu_G_iRf**2 / (mu_G_iRf_2 - mu_G_iRf**2)  # Shape parameter of G_iRc
theta_G_iRf = (mu_G_iRf_2 - mu_G_iRf**2) / mu_G_iRf  # Scale parameter of G_iRc

k_G_iRf_hat = mu_G_iRf_hat**2 / (
    mu_G_iRf_hat_2 - mu_G_iRf_hat**2
)  # Shape parameter of G_iRc
theta_G_iRf_hat = (
    mu_G_iRf_hat_2 - mu_G_iRf_hat**2
) / mu_G_iRf_hat  # Scale parameter of G_iRc

Effective Channel Gain $Z_{i,f} = |H_{i,f}|^2 = (h_{i,f} + G_{i,R,f})^2$

In [ ]:
mu_Z_if = (
    fun_mu_G_iRf(2, m_iR, omega_iR, m_Rf, omega_Rf, K // 2, beta_t)
    + fun_mu_h_if(2, m_if, omega_if)
    + (
        2
        * fun_mu_G_iRf(1, m_iR, omega_iR, m_Rf, omega_Rf, K // 2, beta_t)
        * fun_mu_h_if(1, m_if, omega_if)
    )
)

mu_Z_if_2 = (
    fun_mu_G_iRf(4, m_iR, omega_iR, m_Rf, omega_Rf, K // 2, beta_t)
    + fun_mu_h_if(4, m_if, omega_if)
    + (
        6
        * fun_mu_G_iRf(2, m_iR, omega_iR, m_Rf, omega_Rf, K // 2, beta_t)
        * fun_mu_h_if(2, m_if, omega_if)
    )
    + (
        4
        * fun_mu_h_if(3, m_if, omega_if)
        * fun_mu_G_iRf(1, m_iR, omega_iR, m_Rf, omega_Rf, K // 2, beta_t)
    )
    + (
        4
        * fun_mu_h_if(1, m_if, omega_if)
        * fun_mu_G_iRf(3, m_iR, omega_iR, m_Rf, omega_Rf, K // 2, beta_t)
    )
)

mu_Z_if_hat = (
    fun_mu_G_iRf(2, m_iR_hat, omega_iR_hat, m_Rf, omega_Rf, K // 2, beta_t)
    + fun_mu_h_if(2, m_if_hat, omega_if_hat)
    + (
        2
        * fun_mu_G_iRf(1, m_iR_hat, omega_iR_hat, m_Rf, omega_Rf, K // 2, beta_t)
        * fun_mu_h_if(1, m_if_hat, omega_if_hat)
    )
)

mu_Z_if_hat_2 = (
    fun_mu_G_iRf(4, m_iR_hat, omega_iR_hat, m_Rf, omega_Rf, K // 2, beta_t)
    + fun_mu_h_if(4, m_if_hat, omega_if_hat)
    + (
        6
        * fun_mu_G_iRf(2, m_iR_hat, omega_iR_hat, m_Rf, omega_Rf, K // 2, beta_t)
        * fun_mu_h_if(2, m_if_hat, omega_if_hat)
    )
    + (
        4
        * fun_mu_h_if(3, m_if_hat, omega_if_hat)
        * fun_mu_G_iRf(1, m_iR_hat, omega_iR_hat, m_Rf, omega_Rf, K // 2, beta_t)
    )
    + (
        4
        * fun_mu_h_if(1, m_if_hat, omega_if_hat)
        * fun_mu_G_iRf(3, m_iR_hat, omega_iR_hat, m_Rf, omega_Rf, K // 2, beta_t)
    )
)

k_Z_if = mu_Z_if**2 / (mu_Z_if_2 - mu_Z_if**2)  # Shape parameter of Z_if
theta_Z_if = (mu_Z_if_2 - mu_Z_if**2) / mu_Z_if  # Scale parameter of Z_if

k_Z_if_hat = mu_Z_if_hat**2 / (
    mu_Z_if_hat_2 - mu_Z_if_hat**2
)  # Shape parameter of Z_if_hat
theta_Z_if_hat = (
    mu_Z_if_hat_2 - mu_Z_if_hat**2
) / mu_Z_if_hat  # Scale parameter of Z_if_hat

$\mathcal{V}_{f}=\zeta_{i,f}\rho|\textbf{H}_{i, f}|^2 + \zeta_{i',f}\rho|\textbf{H}_{i', f}|^2$

In [ ]:
simulation_V_f = zeta_if * rho * H_if**2 + zeta_if_hat * rho * H_if_hat**2

In [ ]:
mu_V_f = (zeta_if * rho * mu_Z_if) + (
    zeta_if_hat * rho * mu_Z_if_hat
)  # 1st moment of V_f

mu_V_f_2 = rho**2 * (
    (zeta_if * k_Z_if * theta_Z_if) ** 2
    + (zeta_if_hat**2 * k_Z_if_hat * (1 + k_Z_if_hat) * theta_Z_if_hat**2)
    + (
        zeta_if
        * k_Z_if
        * theta_Z_if
        * (zeta_if * theta_Z_if + 2 * zeta_if_hat * k_Z_if_hat * theta_Z_if_hat)
    )
)  # 2nd moment of V_f

k_V_f = mu_V_f**2 / (mu_V_f_2 - mu_V_f**2)  # Shape parameter of V_f
theta_V_f = (mu_V_f_2 - mu_V_f**2) / mu_V_f  # Scale parameter of V_f

$\mathcal{B}_{f}=\zeta_{i,c}\rho|\textbf{H}_{i, f}|^2 + \zeta_{i',c}\rho|\textbf{H}_{i', f}|^2 + 1$

$\mathcal{B}_{f}=\mathcal{W}_{f} + 1$

$\mathcal{W}_{f}=\zeta_{i,c}\rho|\textbf{H}_{i, f}|^2 + \zeta_{i',c}\rho|\textbf{H}_{i', f}|^2$

In [ ]:
simulation_W_f = zeta_ic * rho * H_if**2 + zeta_ic_hat * rho * H_if_hat**2
simulation_B_f = simulation_W_f + 1

In [ ]:
mu_W_f = (zeta_ic * rho * mu_Z_if) + (
    zeta_ic_hat * rho * mu_Z_if_hat
)  # 1st moment of W_f

mu_W_f_2 = rho**2 * (
    (zeta_ic * k_Z_if * theta_Z_if) ** 2
    + (zeta_ic_hat**2 * k_Z_if_hat * (1 + k_Z_if_hat) * theta_Z_if_hat**2)
    + (
        zeta_ic
        * k_Z_if
        * theta_Z_if
        * (zeta_ic * theta_Z_if + 2 * zeta_ic_hat * k_Z_if_hat * theta_Z_if_hat)
    )
)  # 2nd moment of W_f

k_W_f = mu_W_f**2 / (mu_W_f_2 - mu_W_f**2)  # Shape parameter of W_f
theta_W_f = (mu_W_f_2 - mu_W_f**2) / mu_W_f  # Scale parameter of W_f

In [ ]:
mu_B_f = mu_W_f + 1  # 1st moment of B_f
mu_B_f_2 = mu_W_f_2 + 2 * mu_W_f + 1  # 2nd moment of B_f

k_B_f = mu_B_f**2 / (mu_B_f_2 - mu_B_f**2)  # Shape parameter of B_f
theta_B_f = (mu_B_f_2 - mu_B_f**2) / mu_B_f  # Scale parameter of B_f

In [ ]:
simulation_SINR_f = simulation_V_f / simulation_B_f

## Ergodic Rates

$\mathcal{R(k,m,\theta,\Omega)}=\frac{1}{\Omega  \log (2) \Gamma (k) \Gamma (m)}{\pi  \csc (\pi  m) \left(\Omega  \Gamma (k+m) B_{\frac{\theta}{\Omega }}(m,-k-m+1)-\theta  k \Gamma (k) _3\tilde{F}_2\left(1,1,k+1;2,2-m;\frac{\theta }{\Omega}\right)\right)}$

In [ ]:
def rate_matica(k, m, theta, omega):
    z = theta / omega

    return np.real(
        (
            (np.pi * 1 / (np.sin(np.pi * m)))
            * (
                (
                    omega
                    * mpm.gamma(k + m)
                    * (-mpm.betainc(m, (-1 * k) - (1 * m) + 1, theta / omega, 0))
                    - (
                        theta
                        * k
                        * mpm.gamma(k)
                        * (
                            mpm.hyper(
                                [1, 1, 1 + k], [2, 2 - mpm.mpf(m)], z
                            )
                            / (mpm.gamma(2) * mpm.gamma(2 - mpm.mpf(m)))
                        )
                    )
                )
            )
        )
        / (omega * np.log(2) * mpm.gamma(mpm.mpf(k)) * mpm.gamma(mpm.mpf(m)))
    )

$\mathcal{R(k,m,\theta,\Omega)}=\frac{1}{{\log (2) \Gamma (k+m) B(k,m)}}{G_{3,3}^{3,2}\left(\frac{\Omega }{\theta }|
\begin{array}{c}
 0,1-m,1 \\
 0,0,k \\
\end{array}
\right)}$

In [ ]:
def rate_meijerg(k, m, theta, omega):
    return (1 / (mpm.log(2) * mpm.beta(k, m) * mpm.gamma(k + m))) * mpm.meijerg(
        [[0, 1 - m], [1]], [[0, 0, k], []], mpm.mpf(omega) / mpm.mpf(theta)
    )

In [ ]:
mpm.mp.dps = 25

rate_ic = np.zeros_like(rho)
rate_icH = np.zeros_like(rho)
rate_f = np.zeros_like(rho)
rate_ic_u2 = np.zeros_like(rho)
rate_icH_u2 = np.zeros_like(rho)

for i in range(len(rho)):
    rate_ic[i] = rate_meijerg(k_V_ic[i], k_B_ic[i], theta_V_ic[i], theta_B_ic[i])
    rate_icH[i] = rate_meijerg(k_V_ic[i], k_W_ic[i], theta_V_ic[i], theta_W_ic[i])
    rate_icH_u2[i] = rate_meijerg(
        k_V_ic_u2[i], k_W_ic_u2[i], theta_V_ic_u2[i], theta_W_ic_u2[i]
    )
    rate_ic_u2[i] = rate_meijerg(
        k_V_ic_u2[i], k_B_ic_u2[i], theta_V_ic_u2[i], theta_B_ic_u2[i]
    )
    rate_f[i] = rate_meijerg(k_V_f[i], k_B_f[i], theta_V_f[i], theta_B_f[i])

plt.plot(Pt, rate_icH, label="High SNR Asymptote $R_{ic}$", linestyle="--")
plt.plot(Pt, rate_ic, label="Analytical $R_{ic}$")
plt.plot(Pt, rate_ic_u2, label="Analytical $R_{ic_{u2}}$")
plt.plot(Pt, rate_icH_u2, label="High SNR Asymptote $R_{ic_{u2}}$")
plt.plot(Pt, rate_f, label="Analytical $R_f$")
plt.plot(
    Pt,
    np.log2(1 + np.mean(simulation_SINR_ic, axis=0)) - scale_ic,
    linestyle="",
    marker="o",
    markevery=10,
    label="Simulation $R_{ic}$",
)
plt.plot(
    Pt,
    np.log2(1 + np.mean(simulation_SINR_ic_u2, axis=0)) - scale_ic,
    linestyle="",
    marker="o",
    markevery=10,
    label="Simulation $R_{ic_{u2}}$",
)
plt.plot(
    Pt,
    np.log2(1 + np.mean(simulation_SINR_f, axis=0)) + scale_f,
    linestyle="",
    marker="d",
    markevery=10,
    label="Simulation $R_f$",
)
plt.xlabel("Transmit Power per BS $P_t$ [dB]")
plt.ylabel("Rate [bits/s/Hz]")
# place legend on best position
plt.legend(loc="upper left")
plt.show()

In [ ]:
np.save(f"rate_f_{K}", rate_f)

In [ ]:
rate_f_32 = np.load("rate_f_32.npy")
rate_f_70 = np.load("rate_f_70.npy")

plt.plot(Pt, rate_icH, label="High SNR Asymptote $R_{ic}$", linestyle="--")
plt.plot(Pt, rate_ic, label="Analytical $R_{ic}$")
plt.plot(Pt, rate_ic_u2, label="Analytical $R_{ic_{u2}}$")
plt.plot(Pt, rate_icH_u2, label="High SNR Asymptote $R_{ic_{u2}}$")
plt.plot(Pt, rate_f_32, label="Analytical $R_{f_{32}}$")
plt.plot(Pt, rate_f_70, label="Analytical $R_{f_{70}}$")
plt.plot(Pt, rate_f, label="Analytical $R_f$")
plt.xlabel("Transmit Power per BS $P_t$ [dB]")
plt.ylabel("Rate [bits/s/Hz]")
# place legend on best position
plt.legend(loc="upper left")
plt.show()

## Outage Probabilities

$Pr(\lambda_{f}\lt\lambda_{th_f})=\frac{1}{{k B(k,m)}}{\left(\frac{2^{\lambda_{th} /10} \Omega }{\theta }\right)^k 
   {_2F_1\left(k,k+m;k+1;-\frac{2^{\lambda_{th} /10} \Omega }{\theta
   }\right)}}$

In [ ]:
def outage_lt(k_V_ic, k_B_ic, theta_V_ic, theta_B_ic, lambda_th):
    return (
        (((2 ** (lambda_th / 10) * theta_B_ic) / theta_V_ic) ** k_V_ic)
        * mpm.hyp2f1(
            k_V_ic,
            k_B_ic + k_V_ic,
            k_V_ic + 1,
            -((2 ** (lambda_th / 10) * theta_B_ic) / theta_V_ic),
        )
        / (k_V_ic * mpm.beta(k_V_ic, k_B_ic))
    )

$Pr(\lambda_{i,c\rightarrow f}\gt\lambda_{th_f}, \lambda_{i,c}\lt\lambda_{th_c})=1-\left(1-\frac{\left(\frac{\theta _{B_{\text{ic}}}
   2^{\frac{\lambda _{\text{th}_c}}{10}}}{\theta_{V_{\text{ic}}}}\right)^{k_{V_{\text{ic}}}}{_2F_1}\left(k_{V_{\text{ic}}},k_{B_{\text{ic}}}+k_{V_{\text{ic}}};k_{V_{\text{ic}}}+1;-\frac{2^{\frac{\lambda _{\text{th}_c}}{10}} \theta_{B_{\text{ic}}}}{\theta_{V_{\text{ic}}}}\right)}{k_{V_{\text{ic}}}B\left(k_{V_{\text{ic}}},k_{B_{\text{ic}}}\right)}\right) \left(1-\frac{\Gamma\left(k_{B_{\text{icf}}}\right)-\Gamma\left(k_{B_{\text{icf}}}+k_{V_{\text{icf}}}\right)\left(\frac{\theta _{B_{\text{icf}}} 2^{\frac{\lambda_{\text{th}_f}}{10}}}{\theta_{V_{\text{icf}}}}\right)^{k_{V_{\text{icf}}}}{_2\tilde{F}_1}\left(k_{V_{\text{icf}}},k_{B_{\text{icf}}}+k_{V_{\text{icf}}};k_{V_{\text{icf}}}+1;-\frac{2^{\frac{\lambda _{\text{th}_f}}{10}} \theta
   _{B_{\text{icf}}}}{\theta_{V_{\text{icf}}}}\right)}{\Gamma\left(k_{B_{\text{icf}}}\right)}\right)$

In [ ]:
def outage_clt(
    k_V_icf,
    k_B_icf,
    theta_V_icf,
    theta_B_icf,
    k_V_ic,
    k_B_ic,
    theta_V_ic,
    theta_B_ic,
    lambda_th_f,
    lambda_th_c,
):
    return (
        (((2 ** (lambda_th_c / 10) * theta_B_ic) / theta_V_ic) ** k_V_ic)
        * mpm.hyp2f1(
            k_V_ic,
            k_B_ic + k_V_ic,
            k_V_ic + 1,
            -((2 ** (lambda_th_c / 10) * theta_B_ic) / theta_V_ic),
        )
        / (k_V_ic * mpm.beta(k_V_ic, k_B_ic))
    ) * (
        (
            gamma(k_B_icf)
            - (
                gamma(k_B_icf + k_V_icf)
                * (((2 ** (lambda_th_f / 10) * theta_B_icf) / theta_V_icf) ** k_V_icf)
                * (
                    (
                        mpm.hyp2f1(
                            k_V_icf,
                            k_B_icf + k_V_icf,
                            k_V_icf + 1,
                            -((2 ** (lambda_th_f / 10) * theta_B_icf) / theta_V_icf),
                        )
                    )
                    / (gamma(k_V_icf + 1))
                )
            )
        )
        / (gamma(k_B_icf))
    ) + (
        (((2 ** (lambda_th_f / 10) * theta_B_icf) / theta_V_icf) ** k_V_icf)
        * mpm.hyp2f1(
            k_V_icf,
            k_B_icf + k_V_icf,
            k_V_icf + 1,
            -((2 ** (lambda_th_f / 10) * theta_B_icf) / theta_V_icf),
        )
        / (k_V_icf * mpm.beta(k_V_icf, k_B_icf))
    )

In [ ]:
# fmt: off
outage_ic = qfunc(
    (np.mean(pow2db(sp.stats.betaprime.rvs(k_V_ic, k_B_ic, loc=0, scale=theta_V_ic / theta_B_ic,
                                           size=(N*20, len(rho)))) - (-N0) - (-110), axis=0) / 6
))
simulation_outage_ic = qfunc(
    (np.mean(pow2db(simulation_SINR_ic) - (-N0) - (-110), axis=0) / 6)
)
simulation_outage_ic_u2 = qfunc(
    (np.mean(pow2db(simulation_SINR_ic_u2) - (-N0) - (-110), axis=0) / 6)
)
outage_ic_u2 = qfunc(
    (np.mean(pow2db(sp.stats.betaprime.rvs(k_V_ic_u2, k_B_ic_u2, loc=0, scale=theta_V_ic_u2 / theta_B_ic_u2, size=(N*20, len(rho)))) - (-N0) - (-110), axis=0) / 6
))
outage_f = qfunc(
    (np.mean(pow2db(sp.stats.betaprime.rvs(k_V_f, k_B_f, loc=0, scale=theta_V_f / theta_B_f, 
                                           size=(N*20, len(rho)))) - (-N0) - (-110), axis=0) / 6
))
simulation_outage_f = qfunc(
    (np.mean(pow2db(simulation_SINR_f) - (-N0) - (-110), axis=0) / 6)
)
# fmt: on

plt.semilogy(Pt, outage_ic, label="Analytical Pr$_{ic}$")
plt.semilogy(Pt, outage_ic_u2, label="Analytical Pr$_{ic_{u2}}$")
plt.semilogy(Pt, outage_f, label="Analytical Pr$_f$")
plt.semilogy(
    Pt,
    simulation_outage_ic,
    linestyle="",
    marker="o",
    markevery=10,
    label="Simulation Pr$_{ic}$",
)
plt.semilogy(
    Pt,
    simulation_outage_ic_u2,
    linestyle="",
    marker="d",
    markevery=10,
    label="Simulation Pr$_{ic_{u2}}$",
)
plt.semilogy(
    Pt,
    simulation_outage_f,
    linestyle="",
    marker="d",
    markevery=10,
    label="Simulation Pr$_{f}$",
)
# plt.ylim(1e-3, None)
plt.legend(loc="lower left")
plt.xlabel("Transmit Power per BS $P_t$ [dB]")
plt.ylabel("Outage Probability")
plt.show()

In [ ]:
np.save(f"outage_f_{K}.npy", simulation_outage_f)

In [ ]:
outage_f_32 = np.load("outage_f_32.npy")
outage_f_70 = np.load("outage_f_70.npy")

plt.semilogy(Pt, outage_ic, label="Analytical Pr$_{ic}$")
plt.semilogy(Pt, outage_ic_u2, label="Analytical Pr$_{ic_{u2}}$")
plt.semilogy(Pt, outage_f, label="Analytical Pr$_f$")
plt.semilogy(Pt, outage_f_32, label="Simulation Pr$_{f_{32}}$")
plt.semilogy(Pt, outage_f_70, label="Simulation Pr$_{f_{70}}$")
# plt.ylim(1e-3, None)
plt.legend(loc="lower left")
plt.xlabel("Transmit Power per BS $P_t$ [dB]")
plt.ylabel("Outage Probability")
plt.show()